In [1]:
# 前馈神经网络。
######################################

In [2]:
'''
这个案例是使用sklearn内置的多层神经网络，进行时装图像的分类；图像的特征进行了标准化处理。
'''
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


train = pd.read_csv('datasets/fashionmnist/fashion-mnist_train.csv')
test = pd.read_csv('datasets/fashionmnist/fashion-mnist_test.csv')

y_train = train['label']
y_test = test['label']
X_train = train.drop('label', axis=1)
X_test = test.drop('label', axis=1)

#这里对图像的特征进行标准化处理；尽量保证特征均值为0，方差为1。
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

mlpc = MLPClassifier(hidden_layer_sizes=(300, 10), verbose=True, max_iter=50, activation='relu', momentum=0.9, batch_size=256)
mlpc.fit(X_train, y_train)

y_pred = mlpc.predict(X_test)

print '分类准确率为：%f' % accuracy_score(y_test, y_pred)

/Users/michael/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/michael/anaconda/lib/python2.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/michael/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:22: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.


Iteration 1, loss = 0.64891610
Iteration 2, loss = 0.36036537
Iteration 3, loss = 0.31196546
Iteration 4, loss = 0.28178653
Iteration 5, loss = 0.25920956
Iteration 6, loss = 0.23914386
Iteration 7, loss = 0.22273256
Iteration 8, loss = 0.21029295
Iteration 9, loss = 0.19518701
Iteration 10, loss = 0.18419156
Iteration 11, loss = 0.17073466
Iteration 12, loss = 0.16279051
Iteration 13, loss = 0.15433216
Iteration 14, loss = 0.14476532
Iteration 15, loss = 0.13227259
Iteration 16, loss = 0.12629775
Iteration 17, loss = 0.11875410
Iteration 18, loss = 0.11049637
Iteration 19, loss = 0.10680316
Iteration 20, loss = 0.11000878
Iteration 21, loss = 0.10391220
Iteration 22, loss = 0.09106821
Iteration 23, loss = 0.08868570
Iteration 24, loss = 0.08439613
Iteration 25, loss = 0.07733864
Iteration 26, loss = 0.07291795
Iteration 27, loss = 0.07171856
Iteration 28, loss = 0.07061174
Iteration 29, loss = 0.06284936
Iteration 30, loss = 0.05995603


/Users/michael/anaconda/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (30) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


分类准确率为：0.886300


In [3]:
##############################

In [4]:
'''
采用TensorFlow，自行手动构建一个前馈神经网络层模块，进行时装图像的分类。
'''
import tensorflow as tf
        

#自行手动构建一个前馈神经网络层模块。
def ffn_layer(X, n_outputs, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2.0 / np.sqrt(n_inputs + n_outputs)
        init = tf.truncated_normal((n_inputs, n_outputs), stddev=stddev)
        W = tf.Variable(init, name='weights')
        b = tf.Variable(tf.zeros([n_outputs]), name='bias')
        
        V = tf.matmul(X, W) + b
        
        if activation:
            return activation(V)
        else:
            return V
        
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

train = pd.read_csv('datasets/fashionmnist/fashion-mnist_train.csv')
test = pd.read_csv('datasets/fashionmnist/fashion-mnist_test.csv')

y_train = train['label'].values
y_test = test['label'].values

X_train = train.drop('label', axis=1)
X_test = test.drop('label', axis=1)

ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

train_samples, n_features = np.shape(X_train)
test_samples = len(X_test)

n_inputs = n_features
n_hidden = 300
n_outputs = 10
batch_size = 256
n_batches = train_samples / batch_size
n_epochs = 50

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int32, shape=(None), name='y')

hidden = ffn_layer(X, n_hidden, name='hidden', activation=tf.nn.relu)
logits = ffn_layer(hidden, n_outputs, name='outputs')

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)

loss = tf.reduce_mean(xentropy, name='loss')

learning_rate = 1e-3

optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)

training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)

accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    for iteration in range(n_epochs):
        for i in range(n_batches):
            np.random.seed(iteration * n_batches + i)
            indices = np.random.randint(train_samples, size=batch_size)
            X_batch = X_train[indices]
            y_batch = y_train[indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        print 'Iteration %d, loss = %f' % (iteration + 1, loss.eval(feed_dict = {X: X_train, y: y_train}))
    print '分类准确率为：%f' % accuracy.eval(feed_dict = {X: X_test, y: y_test})

/Users/michael/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/michael/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:39: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.


Iteration 1, loss = 0.577067
Iteration 2, loss = 0.496639
Iteration 3, loss = 0.458209
Iteration 4, loss = 0.435095
Iteration 5, loss = 0.414758
Iteration 6, loss = 0.401343
Iteration 7, loss = 0.389962
Iteration 8, loss = 0.381411
Iteration 9, loss = 0.370728
Iteration 10, loss = 0.364026
Iteration 11, loss = 0.357159
Iteration 12, loss = 0.350565
Iteration 13, loss = 0.345582
Iteration 14, loss = 0.339722
Iteration 15, loss = 0.334863
Iteration 16, loss = 0.329999
Iteration 17, loss = 0.325815
Iteration 18, loss = 0.322130
Iteration 19, loss = 0.318093
Iteration 20, loss = 0.314374
Iteration 21, loss = 0.310942
Iteration 22, loss = 0.308298
Iteration 23, loss = 0.304303
Iteration 24, loss = 0.300820
Iteration 25, loss = 0.298250
Iteration 26, loss = 0.295116
Iteration 27, loss = 0.292166
Iteration 28, loss = 0.289828
Iteration 29, loss = 0.287523
Iteration 30, loss = 0.284579
分类准确率为：0.881500


In [5]:
##############################

In [6]:
'''
采用TensorFlow内置的前馈神经网络模块，自行手动构建一个前馈神经网络层模块，进行时装图像的分类。
'''
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

train = pd.read_csv('datasets/fashionmnist/fashion-mnist_train.csv')
test = pd.read_csv('datasets/fashionmnist/fashion-mnist_test.csv')

y_train = train['label'].values
y_test = test['label'].values

X_train = train.drop('label', axis=1)
X_test = test.drop('label', axis=1)

ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

train_samples, n_features = np.shape(X_train)
test_samples = len(X_test)

import tensorflow as tf
        
n_inputs = n_features
n_hidden = 300
n_outputs = 10
batch_size = 256
n_batches = train_samples / batch_size
n_epochs = 50


X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int32, shape=(None), name='y')

#采用TensorFlow内置的前馈神经网络模块，搭建网络。
hidden = tf.layers.dense(X, n_hidden, name='hidden', activation=tf.nn.relu)
logits = tf.layers.dense(hidden, n_outputs, name='outputs')

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)

loss = tf.reduce_mean(xentropy, name='loss')

learning_rate = 1e-3

optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)

training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)

accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    
    for iteration in range(n_epochs):
        for i in range(n_batches):
            np.random.seed(iteration * n_batches + i)
            indices = np.random.randint(train_samples, size=batch_size)
            X_batch = X_train[indices]
            y_batch = y_train[indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        print 'Iteration %d, loss = %f' % (iteration + 1, loss.eval(feed_dict = {X: X_train, y: y_train}))
    
    saver.save(sess, 'models/section_3.2/my_final_model.ckpt')
    
    print '分类准确率为：%f' % accuracy.eval(feed_dict = {X: X_test, y: y_test})
    

/Users/michael/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:20: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.


Iteration 1, loss = 0.590411
Iteration 2, loss = 0.505205
Iteration 3, loss = 0.465082
Iteration 4, loss = 0.441471
Iteration 5, loss = 0.421341
Iteration 6, loss = 0.407483
Iteration 7, loss = 0.395882
Iteration 8, loss = 0.386541
Iteration 9, loss = 0.376986
Iteration 10, loss = 0.369645
Iteration 11, loss = 0.363313
Iteration 12, loss = 0.357098
Iteration 13, loss = 0.351824
Iteration 14, loss = 0.346092
Iteration 15, loss = 0.341439
Iteration 16, loss = 0.336669
Iteration 17, loss = 0.332495
Iteration 18, loss = 0.328933
Iteration 19, loss = 0.324563
Iteration 20, loss = 0.320958
Iteration 21, loss = 0.317798
Iteration 22, loss = 0.314866
Iteration 23, loss = 0.311352
Iteration 24, loss = 0.307879
Iteration 25, loss = 0.305185
Iteration 26, loss = 0.302094
Iteration 27, loss = 0.299294
Iteration 28, loss = 0.296721
Iteration 29, loss = 0.294581
Iteration 30, loss = 0.291754
分类准确率为：0.881300


In [7]:
'''
采用TensorFlow内置的模型存储功能，复用之前训练好的神经网络结构和参数。
'''
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

train = pd.read_csv('datasets/fashionmnist/fashion-mnist_train.csv')
test = pd.read_csv('datasets/fashionmnist/fashion-mnist_test.csv')

y_train = train['label'].values
y_test = test['label'].values

X_train = train.drop('label', axis=1)
X_test = test.drop('label', axis=1)

ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

train_samples, n_features = np.shape(X_train)
test_samples = len(X_test)

import tensorflow as tf

# 读取神经网络结构（图）。
saver = tf.train.import_meta_graph('models/section_3.2/my_final_model.ckpt.meta')

with tf.Session() as sess:
    # 加载所有相关参数。
    saver.restore(sess, 'models/section_3.2/my_final_model.ckpt')
    
    # 重现分类准确率。
    print '分类准确率为：%f' % accuracy.eval(feed_dict = {X: X_test, y: y_test})
    

/Users/michael/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:20: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.


INFO:tensorflow:Restoring parameters from models/section_3.2/my_final_model.ckpt
分类准确率为：0.881300
